In [ ]:
%pip install langchain-google-cloud-sql-pg langchain-google-vertexai pdfplumber python-dotenv

Defaulting to user installation because normal site-packages is not writeable
  Using cached google_cloud_core-2.4.2-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached google_resumable_media-2.7.2-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached google_crc32c-1.6.0-cp312-cp312-win_amd64.whl.metadata (2.4 kB)
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   ------------------ --------------------- 0.5/1.2 MB 1.3 MB/s eta 0:00:01
   ------------------ --------------------- 0.5/1.2 MB 1.3 MB/s eta 0:00:01
   ------------------------------------ --- 1.0/1.2 MB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   - -------------------------------------- 0.3/5.6 MB ? eta -:--:--
   ----- ---------------------------------- 0.8/5.6 MB 2.2 MB/s eta 0:00:03
   --------- -------------

In [ ]:
# Imports
import os
import time
from dotenv import load_dotenv
from sqlalchemy.exc import ProgrammingError
import pdfplumber
from langchain_core.documents import Document
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_cloud_sql_pg import PostgresEngine, PostgresVectorStore


In [20]:
# Load environment variables
load_dotenv()

True

In [ ]:
from config import PROJECT_ID, REGION, INSTANCE, DATABASE, DB_USER
DB_PASSWORD = os.environ["DB_PASSWORD"]
TABLE_NAME = "project_table"  

In [ ]:
!gcloud auth login

In [ ]:
# Step 1: Connect to the PostgreSQL database using langchain
from langchain_google_cloud_sql_pg import PostgresEngine
engine = PostgresEngine.from_instance(
    project_id=PROJECT_ID,
    instance=INSTANCE,
    region=REGION,
    database=DATABASE,
    user=DB_USER,
    password=DB_PASSWORD,
)

In [ ]:
# Step 2: Create the table for storing documents if not exists
try:
    await engine.ainit_vectorstore_table(
        table_name=TABLE_NAME,
        vector_size=768  
    )
except ProgrammingError:
    print("Table already created")

In [ ]:
# Step 3: Extract text from PDF
# Function to extract text from a PDF and return a list of Document objects
def extract_pdf_content(pdf_path):
    documents = []
    with pdfplumber.open(pdf_path) as pdf:
        for page_number, page in enumerate(pdf.pages, start=1):
            text = page.extract_text()  # Extract text from the page
            if text:  # If the page contains text
                doc_metadata = {"page_number": page_number, "source": pdf_path}
                doc = Document(metadata=doc_metadata, page_content=text)
                documents.append(doc)
    return documents


pdf_path = "Global_ESG_Q4_2024_Flow_Report_FINAL.pdf"
documents = extract_pdf_content(pdf_path)

# Print extracted documents with metadata
for doc in documents:
    print(f"Page Number: {doc.metadata['page_number']}")
    print(f"Content:\n{doc.page_content}\n")

Page Number: 1
Content:
Global Sustainable Fund Flows: Q4 2024 in Review
Despite headwinds , inflows increase, driven by Europe.
Morningstar Sustainalytics Key Takeaways
27 January 2025
× Global sustainable open-end and exchange-traded funds ended 2024 with the highest quarterly inflows
of the year, with subscriptions amounting to USD 16 billion in the fourth quarter, a notable uptick from
Contents the restated inflows of USD 9.2 billion in the third quarter. However, over the full year, inflows into
1 Key Takeaways
global sustainable funds shrank by half, while the rest of the market enjoyed a boom.
1 The Global Sustainable Fund Universe
8 Europe × Europe drove the higher inflows in the fourth quarter, garnering USD 18.5 billion, more than doubling
23 United States
the restated USD 8.9 billion in the previous quarter.
34 Canada
36 Australia and New Zealand × In the US, redemptions in the last quarter slid further to USD 4.3 billion compared with the USD 2.0
40 Japan billion outflow re

In [ ]:
# Step 4: Initialize embeddings and vector store
embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest",
    project=PROJECT_ID
)
vector_store = PostgresVectorStore.create_sync(
    engine=engine,
    table_name=TABLE_NAME,
    embedding_service=embedding,
)

In [ ]:
# Step 5: Add documents to vector store in batches
def add_documents_in_batches(vector_store, documents, batch_size=5):
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i + batch_size]
        try:
            vector_store.add_documents(batch)
            print(f"Added batch {i // batch_size + 1}/{len(documents) // batch_size + 1}")
        except Exception as e:
            print(f"Error adding batch: {e}")
            time.sleep(60)  # Wait and retry if the quota is exhausted

add_documents_in_batches(vector_store, documents, batch_size=5)


Added batch 1/11
Added batch 2/11
Added batch 3/11
Added batch 4/11
Added batch 5/11
Added batch 6/11
Added batch 7/11
Added batch 8/11
Added batch 9/11
Added batch 10/11


In [ ]:
# Step 6: Test query to verify stored embeddings
results = vector_store.similarity_search("sustainable investment", k=3)

for i, result in enumerate(results):
    print(f"Result {i + 1}:")
    print(f"Content: {result.page_content[:500]}")
    print(f"Metadata: {result.metadata}\n")


Result 1:
Content: Page 49 of 50 Global Sustainable Fund Flows—Q4 2024 | January 2025 | See Important Disclosures at the end of this report.
Page 49 of 50 Healthcare Observer | 9 February 2025
Page 49 of 50 Paper Title | 9 February 2025
Page 49 of 50 Healthcare Observer | 9 February 2025
Appendix—Defining the Global Sustainable Fund Universe
Page 49 of 50 Paper Title | 9 February 2025
Page 49 of 50
HTehaelt hgcaloreb Oabls esruvesr t|a 9i nFeabbrulaery f2u0n25d universe encompasses open-end funds and ETFs that, by
Metadata: {'page_number': 49, 'source': 'Global_ESG_Q4_2024_Flow_Report_FINAL.pdf'}

Result 2:
Content: Page 4 of 50 Global Sustainable Fund Flows—Q4 2024 | January 2025 | See Important Disclosures at the end of this report.
Page 4 of 50 Healthcare Observer | 9 February 2025
Page 4 of 50 Paper Title | 9 February 2025
Page 4 of 50 Healthcare Observer | 9 February 2025
From the exhibit below, we can see that active strategies suffered the most from the lower appetite for
Page 4

In [ ]:
# Step 7: Query with retriever
query = "tell me about esg ?"
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.5}
)

docs = retriever.invoke(query)

# Output the results
for doc in docs:
    print(doc.metadata, doc.page_content)


{'page_number': 18, 'source': 'Global_ESG_Q4_2024_Flow_Report_FINAL.pdf'} Page 18 of 50 Global Sustainable Fund Flows—Q4 2024 | January 2025 | See Important Disclosures at the end of this report.
Page 18 of 50 Healthcare Observer | 9 February 2025
Page 18 of 50 Paper Title | 9 February 2025
Page 18 of 50 Healthcare Observer | 9 February 2025
We expect activity among sustainable funds to gather momentum over the next five months as EU funds
Page 18 of 50 Paper Title | 9 February 2025
have until May 21, 2025, to comply with ESMA's fund naming guidelines. In addition to protecting
Page 18 of 50 Hinevalethsctaorer sO basegraveirn | s9t F gebrreuearny w202a5s hing risk, the guidelines aim to provide minimum standards for funds that
use specific ESG terms in their names. Last quarter, we identified around 4,700 EU funds with ESG or
Page 18 of 50 Paper Title | 9 February 2025
sustainability-related terms in their names that fall within the scope of the guidelines.
Page 18 of 50 H ealthcare Ob